In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
dataset = pd.read_csv("three_sentiments.csv", index_col='ID')


In [4]:
# Weighted verage of the sentiments from the three
dataset['average_sentiment'] = (dataset['bert_sentiment'] + dataset['vader_sentiment'] + 0.5*dataset['textblob_sentiment'])/2.5
dataset.head()

,interview_date,country,NPS,translated_comment,customer_type,comment_length,translated_comment_preprocessed,bert_sentiment,vader_sentiment,sentiment,textblob_sentiment,average_sentiment
ID,,,,,,,,,,,,
17,2022-07-01,Poland,5.0,High prices,detractors,11,high price,10,5.000,neu,5.800000,6.933333
20,2020-11-22,Sweden,8.0,beat caught,passives,11,beat caught,2,5.000,neu,5.000000,4.000000
29,2022-08-03,Poland,10.0,Very easy and transparent meter takeover proce...,promoters,50,easy transparent meter takeover procedure,10,7.202,pos,7.166667,8.122889
32,2022-07-06,Poland,5.0,confusion in the name of the company,detractors,36,confusion name company,2,3.520,neg,5.000000,3.506667
42,2022-06-23,Romania,7.0,"solving immediate problems, that is, first of ...",passives,218,solving immediate problem first malfunction oc...,6,3.194,neg,6.250000,5.148000


In [5]:
# Perform a correlation analysis between average sentiment score and NPS
correlation = dataset[['NPS', 'average_sentiment']].corr()

# Calculate the standard deviation and standard error of the difference between NPS and average sentiment score
dataset['difference'] = dataset['NPS'] - dataset['average_sentiment']
std_dev = dataset['difference'].std()
std_error = dataset['difference'].sem()

correlation

,NPS,average_sentiment
NPS,1.000000,0.585067
average_sentiment,0.585067,1.000000


In [6]:
std_dev

2.3013112168693706

In [7]:
std_error

0.010381338671229246

In [8]:
strong_alignment_threshold = std_dev/2.0


def categorize_alignment(diff):
    if abs(diff) <= strong_alignment_threshold:
        return 'Strong Alignment'
    elif abs(diff) > strong_alignment_threshold and abs(diff) < std_dev:
        return 'Moderate Alignment'
    else:
        return 'Weak Alignment'

# Apply the categorization function to each row
dataset['alignment_category'] = dataset['difference'].apply(categorize_alignment)


In [9]:
dataset[['NPS', 'average_sentiment', 'alignment_category']]

,NPS,average_sentiment,alignment_category
ID,,,
17,5.0,6.933333,Moderate Alignment
20,8.0,4.000000,Weak Alignment
29,10.0,8.122889,Moderate Alignment
32,5.0,3.506667,Moderate Alignment
42,7.0,5.148000,Moderate Alignment
...,...,...,...
503873,8.0,7.307788,Strong Alignment
503874,8.0,5.333333,Weak Alignment
503875,10.0,5.935333,Weak Alignment


In [10]:
dataset[dataset['alignment_category'] != "Weak Alignment"].to_csv("cleaned_dataset_without_weakv2.csv")
# dataset.to_csv("alignments.csv")

In [ ]:
df2 = dataset[dataset['alignment_category'] != "Weak Alignment"]

plt.figure(figsize=(10, 6))
plt.scatter(dataset['average_sentiment'], dataset['NPS'])
plt.title(f'Correlation between NPS and Sentiment Scores with all comments')
plt.xlabel('Sentiment Scores')
plt.ylabel('NPS Scores')
plt.show()